In [10]:
import ast
import pandas as pd

In [11]:
# Cargar el archivo CSV
file_path ="C:\\Users\\yanin\\OneDrive\\Desktop\\proyecto 1\\archivos\\movies_dataset (1).csv"



df = pd.read_csv(file_path)


C:\Users\yanin\AppData\Local\Temp\ipykernel_14772\365787197.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [12]:
# Convertir la columna 'popularity' a numérico
df['popularity'] = pd.to_numeric(df['popularity'], errors='coerce')

# Eliminar filas con valores nulos en 'popularity'
df_clean = df.dropna(subset=['popularity']).copy()

# Estratificar por cuartiles de popularidad
df_clean['popularity_bin'] = pd.qcut(df_clean['popularity'], q=4, duplicates='drop')

# Tomar una muestra estratificada de 1000 filas, evitando errores con pocos datos
sample_df = df_clean.groupby('popularity_bin', group_keys=False).apply(
    lambda x: x.sample(min(len(x), 1000 // df_clean['popularity_bin'].nunique()), random_state=42)
).copy()

# Eliminar la columna auxiliar
sample_df = sample_df.drop(columns=['popularity_bin'])

C:\Users\yanin\AppData\Local\Temp\ipykernel_14772\2937174.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sample_df = df_clean.groupby('popularity_bin', group_keys=False).apply(
C:\Users\yanin\AppData\Local\Temp\ipykernel_14772\2937174.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_df = df_clean.groupby('popularity_bin', group_keys=False).apply(


In [13]:

# Convertir la columna 'belongs_to_collection' a diccionarios
sample_df["belongs_to_collection"] = sample_df["belongs_to_collection"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) and pd.notna(x) else None
)

# Desanidar valores en un nuevo DataFrame
df_collection = sample_df["belongs_to_collection"].apply(
    lambda x: {"idCollection": x.get("id"), "nameCollection": x.get("name")} if isinstance(x, dict) else None
).apply(pd.Series)

# Convertir 'idCollection' a Int64 (soporta valores nulos)
df_collection["idCollection"] = df_collection["idCollection"].astype("Int64")

# Verificar valores nulos en release_date antes y después de la conversión
null_dates = df["release_date"].isnull().sum()
df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
null_dates_after = df["release_date"].isnull().sum()

In [14]:
# Extraer el año de la fecha de estreno
df["release_year"] = df["release_date"].dt.year

# Eliminar filas donde 'release_date' sigue siendo nulo
df = df.dropna(subset=['release_date'])

# Verificar el cambio
df[['release_date', 'release_year']].head()


,release_date,release_year
0,1995-10-30,1995.0
1,1995-12-15,1995.0
2,1995-12-22,1995.0
3,1995-12-22,1995.0
4,1995-02-10,1995.0


In [15]:
# Convertir 'revenue' y 'budget' a valores numéricos, reemplazando errores con 0
df["revenue"] = pd.to_numeric(df["revenue"], errors="coerce").fillna(0)
df["budget"] = pd.to_numeric(df["budget"], errors="coerce").fillna(0)

In [16]:
# Calcular el retorno de inversión (ROI)
df["return"] = df.apply(lambda x: x["revenue"] / x["budget"] if x["budget"] > 0 else 0, axis=1)

# Mostrar las primeras filas con la nueva columna
df[["revenue", "budget", "return"]].head()

,revenue,budget,return
0,373554033.0,30000000,12.451801
1,262797249.0,65000000,4.043035
2,0.0,0,0.000000
3,81452156.0,16000000,5.090760
4,76578911.0,0,0.000000


In [17]:
# Eliminar las columnas no utilizadas
columns_to_drop = ["video", "imdb_id", "adult", "original_title", "poster_path", "homepage"]
df = df.drop(columns=columns_to_drop, errors="ignore")

# Verificar que las columnas fueron eliminadas
df.head()


,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,1995.0,12.451801
1,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995.0,4.043035
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.712900,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995.0,0.000000
3,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,1995.0,5.090760
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,1995.0,0.000000


In [18]:



# Guardar el DataFrame modificado en un nuevo archivo CSV
output_file_path = "C:\\Users\\yanin\\OneDrive\\Desktop\\proyecto 1\\archivos\\movies_dataset_modificado.csv"
df.to_csv(output_file_path, index=False)

print(f"Archivo guardado en: {output_file_path}")

Archivo guardado en: C:\Users\yanin\OneDrive\Desktop\proyecto 1\archivos\movies_dataset_modificado.csv
